In [26]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot
%matplotlib inline

In [27]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [28]:
# Load dataset
data = pd.read_csv("diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [33]:
# Get features and target
X = data.drop(['Outcome'], axis = 1)
y = data['Outcome']
X.shape,y.shape

((768, 8), (768,))

In [34]:
#splitting the model into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=101)

In [35]:
#training a logistics regression model
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)
print("Accuracy = "+ str(accuracy_score(y_test,predictions)))

Accuracy = 0.7922077922077922


In [36]:
#defining various steps required for the genetic algorithm
def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)
        chromosome[:int(0.3*n_feat)]=False
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population):
    scores = []
    for chromosome in population:
        logmodel.fit(X_train.iloc[:,chromosome],y_train)
        predictions = logmodel.predict(X_test.iloc[:,chromosome])
        scores.append(accuracy_score(y_test,predictions))
    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)
    return list(scores[inds][::-1]), list(population[inds,:][::-1])

def selection(pop_after_fit,n_parents):
    population_nextgen = []
    for i in range(n_parents):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen

def crossover(pop_after_sel):
    population_nextgen=pop_after_sel
    for i in range(len(pop_after_sel)):
        child=pop_after_sel[i]
        child[3:7]=pop_after_sel[(i+1)%len(pop_after_sel)][3:7]
        population_nextgen.append(child)
    return population_nextgen

def mutation(pop_after_cross,mutation_rate):
    population_nextgen = []
    for i in range(0,len(pop_after_cross)):
        chromosome = pop_after_cross[i]
        for j in range(len(chromosome)):
            if random.random() < mutation_rate:
                chromosome[j]= not chromosome[j]
        population_nextgen.append(chromosome)
    #print(population_nextgen)
    return population_nextgen

def generations(size,n_feat,n_parents,mutation_rate,n_gen,X_train,
                                   X_test, y_train, y_test):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(size,n_feat)
    for i in range(n_gen):
        scores, pop_after_fit = fitness_score(population_nextgen)
        print(scores[:2])
        pop_after_sel = selection(pop_after_fit,n_parents)
        pop_after_cross = crossover(pop_after_sel)
        population_nextgen = mutation(pop_after_cross,mutation_rate)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
    return best_chromo,best_score

In [45]:
chromo,score=generations(size=100,n_feat=8,n_parents=50,mutation_rate=0.10,
                     n_gen=38,X_train=X_train,X_test=X_test,y_train=y_train,y_test=y_test)
logmodel.fit(X_train.iloc[:,chromo[-1]],y_train)
predictions = logmodel.predict(X_test.iloc[:,chromo[-1]])
print("Accuracy score after genetic algorithm is= "+str(accuracy_score(y_test,predictions)))

[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8008658008658008, 0.8008658008658008]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948052, 0.8051948051948052]
[0.8008658008658008, 0.8008658008658008]
[0.8008658008658008, 0.8008658008658008]
[0.8051948051948052, 0.8051948051948052]
[0.8051948051948